In [8]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re #regex \. - dot, \d - digit, \D -non digit, \w - alphanumeric, \W - non alphanumeric

src_dir = r'D:\Ravi_Data_Engineer\02_Target\01_raw'
tgt_dir = r'D:\Ravi_Data_Engineer\02_Target\02_cleansed'
os.makedirs(tgt_dir, exist_ok=True)

def clean_datefields(df):
    # Convert columns that look like dates
    for col in df.columns:
        if "date" in col.lower():
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass  # skip if date conversion fails

    return df

keyword = input("Enter the keyword to search in file names:").lower()

files_list_tgt = [f for f in os.listdir(tgt_dir) if keyword in f.lower()]

if files_list_tgt:
    os.remove(os.path.join(tgt_dir, files_list_tgt[0])) # remove the file if it exists in target directory
    print(f"File {os.path.join(tgt_dir, files_list_tgt[0])} deleted from {tgt_dir}.")

files_list_src = [f for f in os.listdir(src_dir) if keyword in f.lower()]

if not files_list_src:
    print(f"No files found containing the keyword '{keyword}' in {src_dir}.")

cleaned_dfs = []

for file in files_list_src:
    file_path = os.path.join(src_dir, file)
    print(f"\n Reading: {file_path}")

    try:
        df = pd.read_csv(file_path) # read csv file

        df.columns = df.columns.str.strip() # remove leading and trailing spaces from column names

        df.columns = df.columns.str.replace(r'\.\d+$', '', regex=True) # replace the last occurrence of a dot followed by digits in column names
        df = df.loc[:,~df.columns.duplicated()] # remove duplicate columns

        df = df.dropna(how='all') # remove rows where all elements are NaN

        string_cols = df.select_dtypes(include='object').columns # checkign the columns with string data type
        df[string_cols] = df[string_cols].fillna('Unknown') # replacing the nulls with Unknown

        df = df.map(lambda x: x.strip() if isinstance(x, str) else x) # strip leading and trailing spaces from string values in dataframe

        #df['Phone'] = df['Phone'].apply(lambda x: re.sub(r'\D', '', str(x)) if pd.notnull(x) and len(re.sub(r'\D', '', str(x))) == 10 else '1234567890')
        #using lambda, regex to replace find the Phone field and replace per the needed format

        df = clean_datefields(df)

        cleaned_dfs.append((file, df)) # appended to the cleaned_dfs list
        print(f"Cleaned: {file} - {df.shape[0]} rows, {df.shape[1]} columns")

    except Exception as e:
        print(f"Failed to read {file}: {e}")
        continue

for files in cleaned_dfs:
    file_name, df = files
    src_file = os.path.join(src_dir,file_name)
    orig_name = os.path.splitext(file_name)[0]
    ext = os.path.splitext(file_name)[1]
    fn = orig_name.replace('raw_', f'cleansed_')
    new_name = f"{fn}{ext}" # created a new name for the files with prefix as cleansed_
    tgt_file = os.path.join(tgt_dir,new_name)
      
    try:
        df.to_csv(tgt_file, index=False)
        print(f"Saved cleaned data to: {tgt_file}")

    except Exception as e:
        print(f"Failed to save {file_name}: {e}")



File D:\Ravi_Data_Engineer\02_Target\02_cleansed\cleansed_retail_products.csv deleted from D:\Ravi_Data_Engineer\02_Target\02_cleansed.

 Reading: D:\Ravi_Data_Engineer\02_Target\01_raw\raw_retail_products.csv
Cleaned: raw_retail_products.csv - 500 rows, 6 columns
Saved cleaned data to: D:\Ravi_Data_Engineer\02_Target\02_cleansed\cleansed_retail_products.csv
